### Figures for cross-participant regression analysis 
1) Results Figure 3 (average timeseries for within and cross-model with difference, example timeseries)
2) Supplementary Figure S2 (timeseries for all dimensions)


In [ ]:
"""
Author: linateichmann
Email: lina.teichmann@nih.gov

    Created on 2023-03-30 12:39:32
    Modified on 2023-03-30 12:39:32
"""

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import mne, os
%matplotlib qt

bids_dir = '/System/Volumes/Data/misc/data16/teichmanna2/2020_MEG_things/THINGS_biowulf/THINGS-MEG-bids/'
res_folder = f'{bids_dir}/derivatives/meg_paper/output/regression'
font = 'Arial'
text_size = 12
text_size_big = 16
text_size_small = 8

plt.rcParams['font.size'] = text_size
plt.rcParams['font.family'] = font

colours = pd.read_csv(f'{bids_dir}/sourcedata/meg_paper/colors66.txt',header=None)
labels = pd.read_csv(f'{bids_dir}/sourcedata/meg_paper/labels_super_short.txt',header=None)


# load data
def load_epochs(bids_dir,participant,behav):
    epochs = mne.read_epochs(f'{bids_dir}/derivatives/preprocessed/preprocessed_P{participant}-epo.fif',preload=False)
    # THINGS-category number & image number start at 1 (matlab based) so subtracting 1 to use for indexing
    epochs.metadata['things_image_nr'] = epochs.metadata['things_image_nr']-1
    epochs.metadata['things_category_nr'] = epochs.metadata['things_category_nr']-1
    # adding dimensional weights to metadata
    epochs.metadata[['dim'+ str(d+1) for d in range(66)]] = np.nan
    for i in range(len(epochs.metadata)):
        if not np.isnan(epochs.metadata.loc[i,'things_image_nr']):
            epochs.metadata.loc[i,['dim'+ str(d+1) for d in range(66)]] = behav[int(epochs.metadata.loc[i,'things_image_nr']),:]
    return epochs

behav = np.loadtxt(f'{bids_dir}/sourcedata/meg_paper/predictions_66d_elastic_clip-ViT-B-32_visual_THINGS.txt')
epochs = load_epochs(bids_dir,1,behav)

epochs_exp = epochs[(epochs.metadata['trial_type']=='exp')]   
epochs_exp.metadata.reset_index(inplace=True,drop=True)


# load results
all_dat_dims=[]
for p in range(1,5):
    all_dat_dims.append(pd.read_csv(f'{res_folder}/P{p}_linreg_within.csv',index_col=0))
all_dat_dims_cross = pd.read_csv(f'{res_folder}/Linreg_cross_predict-dims.csv',index_col=0)


# change data order based on peak amplitude
avg_dim_data = np.mean(all_dat_dims,axis=0)
sorted_idx = np.flipud(np.argsort(np.max(avg_dim_data,axis=0)))

avg_dim_data_ranked = avg_dim_data[:,sorted_idx]
dim_data_ranked = np.array(all_dat_dims)[:,:,sorted_idx]
colours_ranked = colours.to_numpy()[sorted_idx,:]
labels_ranked = labels.loc[sorted_idx,:]
labels_ranked.reset_index(inplace=True,drop=True)

dim_data_cross_ranked = np.array(all_dat_dims_cross)[:,sorted_idx]

filter_col = [col for col in epochs_exp.metadata.columns if col.startswith('dim')] 
weights_sorted_dims = epochs_exp.metadata.loc[:,filter_col].to_numpy()[:,sorted_idx]



In [140]:
# MAKE RESULTS FIGURE
plt.close('all')
axd = plt.figure(constrained_layout=True,figsize=(8.25, 11.75)).subplot_mosaic(

    """
    .AAAA.
    BBCCDD
    EEFFGG
    ......
    ......
    ......
    ......
    ......
    """)

fig = plt.gcf()

ax = axd['A']
ax.plot(epochs.times*1000, epochs.times*0, 'grey', lw=1, linestyle='--')
ax.plot(epochs.times*1000, np.mean(avg_dim_data_ranked,axis=1),'b',alpha=0.3,lw=2,label='within-participant')
ax.plot(epochs.times*1000, np.mean(dim_data_cross_ranked,axis=1),'b',lw=2,label='cross-participant')

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_title('all dimensions',c='b')
ax.set_xlim([epochs.times[0]*1000,epochs.times[-1]*1000])
ax.legend(frameon=False,fontsize=text_size_small)
ax.set_ylabel('Correlation')
ax.set_xticklabels('')

axin2 = ax.inset_axes([-100,-0.07, 1400,0.05], transform= ax.transData)

diff_data = dim_data_cross_ranked-avg_dim_data_ranked
axin2.plot(epochs.times*1000, np.mean(diff_data,axis=1),'k',lw=2)

axin2.spines['top'].set_visible(False)
axin2.spines['right'].set_visible(False)
axin2.set_title('')
axin2.set_xlim([epochs.times[0]*1000,epochs.times[-1]*1000])

axin2.set_ylabel('Difference')
axin2.set_xlabel('time (ms)')


# add individual timeseries
dim_ranks = [int(i) for i in np.round(np.linspace(0,65,6))]
for rank,ax in zip(dim_ranks,[axd['B'],axd['C'],axd['D'],axd['E'],axd['F'],axd['G']]):

    # plot chance
    ax.plot(epochs.times*1000, epochs.times*0, 'grey', lw=1, linestyle='--')

    ax.plot(epochs.times*1000,avg_dim_data_ranked[:,rank],c=colours_ranked[rank,:],lw=2,alpha=0.3,label='within-participant')
    ax.plot(epochs.times*1000,dim_data_cross_ranked[:,rank],c=colours_ranked[rank,:],lw=2,label='cross-participant')

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_ylim([-0.05,0.3])

    ax.set_title(' ')
    ax.text(700,0.3,labels_ranked.loc[rank,0],c=colours_ranked[rank,:],fontsize=text_size,ha='center')

    ax.set_xlim([epochs.times[0]*1000,epochs.times[-1]*1000])
    ax.legend(frameon=False,fontsize=text_size_small)


axd['B'].text(-400,0.3,' ',fontsize=text_size_big*3)
fig.text(0.01,0.97,' ',fontsize=text_size_big*2)

axd['B'].set_ylabel('Correlation')
axd['E'].set_ylabel('Correlation')

axd['F'].set_xlabel('time (ms)')

# adjust the axis positions for B
left = 0.1
height = 0.1
width = 0.25
top = 0.62
bot = 0.47

right = 1-left-width+0.05
middle = 0.5-width/2+0.025
axd['B'].set_position([left,top,width,height])
axd['C'].set_position([middle,top,width,height])
axd['D'].set_position([right,top,width,height])

axd['E'].set_position([left,bot,width,height])
axd['F'].set_position([middle,bot,width,height])
axd['G'].set_position([right,bot,width,height])


# add A and B labels

fig.text(0.01,0.97,'A',fontsize=text_size_big*2)
fig.text(0.01,0.73,'B',fontsize=text_size_big*2)

fig.savefig(f'{bids_dir}/derivatives/meg_paper/figures/Figure3.pdf')

In [146]:
# supplementary figure showing all dimension timecourses
plt.close('all')
fig,axs = plt.subplots(figsize=(8.25, 11.75),num=2,ncols=6,nrows=11,sharex=True,sharey=True)

axs=axs.flatten()

for i,ax in enumerate(axs):
    ax.plot(epochs.times*1000,epochs.times*0,'grey',linestyle='--')
    
    ax.plot(epochs.times*1000,avg_dim_data_ranked[:,i],c=colours_ranked[i,:],alpha=0.3,lw=2)
    ax.plot(epochs.times*1000,dim_data_cross_ranked[:,i],c=colours_ranked[i,:],alpha=1,lw=2)

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_ylim([-0.05,0.3])


    ax.set_title(labels_ranked.loc[i,0],c='k',fontsize=text_size_small)

    ax.set_xlim([epochs.times[0]*1000,epochs.times[-1]*1000])


fig.supylabel('Correlation: true vs predicted label')
fig.supxlabel('time (ms)')
fig.tight_layout()

fig.savefig(f'{bids_dir}/derivatives/meg_paper/figures/Supplementary_cross_within.pdf')